<a href="https://colab.research.google.com/github/sarithamiryala/DeepLearning/blob/main/pytorch_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [48]:
df = pd.read_csv('/content/drive/MyDrive/DataScience2024/DeepLearning/pytorch/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [49]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [50]:
X = df.iloc[:,3:13]
y = df.iloc[:,13]

In [51]:
X['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [52]:
X['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [53]:
# Create dummy variables for 'Geography' and 'Gender'
geography_dummies = pd.get_dummies(X['Geography'], prefix='Geography', drop_first=True)
gender_dummies = pd.get_dummies(X['Gender'], prefix='Gender', drop_first=True)

# Concatenate the dummy variables with the original DataFrame
X = pd.concat([X, geography_dummies, gender_dummies], axis=1)

# Drop the original 'Geography' and 'Gender' columns
X.drop(['Geography', 'Gender'], axis=1, inplace=True)

In [54]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,False,True,False


In [55]:
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [56]:
##Splitting the dataset into Training set and Test set
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

#scaling

In [57]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [58]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [59]:
y_train

,Exited
7389,0
9275,0
2995,0
5316,0
356,0
...,...
9225,0
4859,0
3264,0
9845,0


In [61]:
type(y_train)

pandas.core.series.Series

# Numpy array to pytorch tensor

In [64]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train.values)
y_test_tensor = torch.from_numpy(y_test.values)

In [66]:
X_train_tensor

tensor([[ 0.1696, -0.4646,  0.0067,  ..., -0.5698,  1.7431, -1.0917],
        [-2.3046,  0.3010, -1.3774,  ...,  1.7549, -0.5737,  0.9160],
        [-1.1912, -0.9431, -1.0314,  ..., -0.5698, -0.5737, -1.0917],
        ...,
        [ 0.9015, -0.3689,  0.0067,  ..., -0.5698, -0.5737,  0.9160],
        [-0.6242, -0.0818,  1.3908,  ..., -0.5698,  1.7431, -1.0917],
        [-0.2840,  0.8753, -1.3774,  ...,  1.7549, -0.5737, -1.0917]],
       dtype=torch.float64)

# Define the model

In [73]:
class ChurnModelNN(torch.nn.Module):
  def __init__(self,X):

    self.weights = torch.rand(X.shape[1],1,dtype=torch.float64, requires_grad=True)
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

  def forward(self,X):
    z = torch.matmul(X,self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    #clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

    #calculate loss
    loss = -torch.mean(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred))
    return loss

# Important Parameter

In [82]:
learning_rate =0.1
epochs = 25


# Training Pipeline

In [83]:
#create model
model = ChurnModelNN(X_train_tensor)

#define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train_tensor)

  # loss calculation
  loss = model.loss_function(y_pred,y_train_tensor)


  # backward pass
  loss.backward()

  # update weights
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss after each epoch
  print(f'Epochs: {epoch + 1},Loss:{loss.item()}')




Epochs: 1,Loss:1.03907296712861
Epochs: 2,Loss:1.021269595406222
Epochs: 3,Loss:1.0038763126519978
Epochs: 4,Loss:0.9868936157613406
Epochs: 5,Loss:0.9703216187969963
Epochs: 6,Loss:0.9541600419237027
Epochs: 7,Loss:0.9384082020871831
Epochs: 8,Loss:0.9230650055709224
Epochs: 9,Loss:0.908128942546695
Epochs: 10,Loss:0.8935980837150281
Epochs: 11,Loss:0.8794700791100725
Epochs: 12,Loss:0.8657421591200377
Epochs: 13,Loss:0.852411137749942
Epochs: 14,Loss:0.8394734181283086
Epochs: 15,Loss:0.8269250002341284
Epochs: 16,Loss:0.8147614907953662
Epochs: 17,Loss:0.8029781152859359
Epochs: 18,Loss:0.7915697319248812
Epochs: 19,Loss:0.7805308475598305
Epochs: 20,Loss:0.7698556352970072
Epochs: 21,Loss:0.7595379537224831
Epochs: 22,Loss:0.7495713675441743
Epochs: 23,Loss:0.739949169471534
Epochs: 24,Loss:0.7306644031401006
Epochs: 25,Loss:0.7217098868811104


In [84]:
model.weights

tensor([[0.2869],
        [0.2118],
        [0.2732],
        [0.5021],
        [0.5402],
        [0.5783],
        [0.2689],
        [0.5677],
        [0.2785],
        [0.1638],
        [0.0615]], dtype=torch.float64, requires_grad=True)

In [85]:
model.bias

tensor([-0.5964], dtype=torch.float64, requires_grad=True)

# Evaluation

In [95]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.85).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy}')



Accuracy: 0.7772700190544128
